# [hotsline](https://github.com/AntoineGrimaldi/hotsline) algorithm to replicate results from [this paper](https://www.techrxiv.org/articles/preprint/A_robust_event-driven_approach_to_always-on_object_recognition/18003077/1)
## Load events of the NMNIST dataset with [Tonic](https://tonic.readthedocs.io/en/latest/index.html)

In [1]:
import tonic, torch, os
%cd ../hots
from utils import get_loader, get_sliced_loader, get_dataset_info, HOTS_Dataset, make_histogram_classification
from network import network
from timesurface import timesurface

print(f' Tonic version installed -> {tonic.__version__}')

transform = tonic.transforms.NumpyAsType(int)
dataset = tonic.datasets.DVSGesture(save_to='../../Data/', train=True, transform=transform)
#get_dataset_info(dataset, properties = ['time', 'mean_isi', 'nb_events']);
print(f'number of samples in the dataset: {len(dataset)}')

/home/antoine/homhots/hotsline/hots
 Tonic version installed -> 1.0.15
number of samples in the dataset: 1077


In [2]:
trainset = tonic.datasets.DVSGesture(save_to='../../Data/', train=True)
testset = tonic.datasets.DVSGesture(save_to='../../Data/', train=False)

transform = tonic.transforms.NumpyAsType(int)

name = 'homeohots'
homeo = True
timestr = '2022-04-22'
dataset_name = 'gesture'

R_first = [2, 4, 8]
N_layers = [2,3,4]
n_first = [8,16]
tau_first = [.1e3,.2e3,.5e3,1e3,2e3,5e3,1e4]

slicing_time_window = 1e6

for lay in N_layers:
    for R in R_first:
        for tau in tau_first:
            for N_neuron in n_first:
                Rz = [R*2**Nl for Nl in range(lay)]
                N_neuronz = [N_neuron*2**Nl for Nl in range(lay)]
                N_pola = N_neuronz.copy()
                N_pola.insert(0,2)
                tauz = [tau*N_pola[Nl] for Nl in range(lay)]
                hots = network(name, dataset_name, timestr, trainset.sensor_size, nb_neurons = N_neuronz, tau = tauz, R = Rz, homeo = homeo)
                filtering_threshold = [2*Rz[L] for L in range(len(Rz))]
                #clustering
                print('clustering')
                loader = get_sliced_loader(trainset, slicing_time_window, dataset_name, True, only_first=True, kfold=10)
                hots.clustering(loader, trainset.ordering, filtering_threshold)
                #training
                print('training')
                loader = get_sliced_loader(trainset, slicing_time_window, dataset_name, True, only_first=True, kfold=2)
                num_sample_train = len(loader)
                hots.coding(loader, trainset.ordering, trainset.classes, filtering_threshold, training=True)
                #testing
                print('testing')
                loader = get_sliced_loader(testset, slicing_time_window, dataset_name, False, only_first=True, kfold=2)
                num_sample_test = len(loader)
                hots.coding(loader, trainset.ordering, trainset.classes, filtering_threshold, training=False)
                
                jitter = (None, None)
                train_path = f'../Records/output/train/{hots.name}_{num_sample_train}_{jitter}/'
                test_path = f'../Records/output/test/{hots.name}_{num_sample_test}_{jitter}/'

                testset_output = HOTS_Dataset(test_path, trainset.sensor_size, transform=transform, dtype=trainset.dtype)
                trainset_output = HOTS_Dataset(train_path, trainset.sensor_size, transform=transform, dtype=trainset.dtype)
                
                score = make_histogram_classification(trainset_output, testset_output, N_neuronz[-1]) 
                print(f' Accuracy: {score*100}%')

device -> cuda
clustering
Read metadata from disk.
training
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/train/2022-04-22_gesture_homeohots_True_[8, 16]_[200.0, 800.0]_[2, 4]_528_(None, None)/
testing
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/test/2022-04-22_gesture_homeohots_True_[8, 16]_[200.0, 800.0]_[2, 4]_132_(None, None)/


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 132/132 [00:00<00:00, 1210.96it/s]


 Accuracy: 8.333333333333332%
device -> cuda
clustering
Read metadata from disk.
training
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/train/2022-04-22_gesture_homeohots_True_[16, 32]_[200.0, 1600.0]_[2, 4]_528_(None, None)/
testing
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/test/2022-04-22_gesture_homeohots_True_[16, 32]_[200.0, 1600.0]_[2, 4]_132_(None, None)/


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 132/132 [00:00<00:00, 1245.81it/s]


 Accuracy: 7.575757575757576%
device -> cuda
clustering
Read metadata from disk.
training
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/train/2022-04-22_gesture_homeohots_True_[8, 16]_[400.0, 1600.0]_[2, 4]_528_(None, None)/
testing
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/test/2022-04-22_gesture_homeohots_True_[8, 16]_[400.0, 1600.0]_[2, 4]_132_(None, None)/


...

 Accuracy: 52.27272727272727%
device -> cuda
clustering
Read metadata from disk.
training
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/train/2022-04-22_gesture_homeohots_True_[16, 32]_[2000.0, 16000.0]_[4, 8]_528_(None, None)/
testing
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/test/2022-04-22_gesture_homeohots_True_[16, 32]_[2000.0, 16000.0]_[4, 8]_132_(None, None)/


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 132/132 [00:00<00:00, 151.70it/s]


 Accuracy: 63.63636363636363%
device -> cuda
clustering
Read metadata from disk.
training
Read metadata from disk.
this dataset have already been processed, check at: 
 ../Records/output/train/2022-04-22_gesture_homeohots_True_[8, 16]_[4000.0, 16000.0]_[4, 8]_528_(None, None)/
testing
Read metadata from disk.


  2%|██▏                                                                                                                                             | 2/132 [00:21<23:04, 10.65s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import copy

def plot_accuracy(scores, parameters, param_to_plot):
    paramlist = ['number of layers', 'R', 'tau', 'number of neurons']
    fig, ax = plt.subplots(figsize=(10,6))
    score_previous = 0
    for ind in range(len(scores)):
        score, tau, n_layer, R, n_neurons = scores[ind], parameters[ind][2], parameters[ind][0], parameters[ind][1], parameters[ind][3]
        if param_to_plot=='tau':
            x = copy.copy(tau)
        elif param_to_plot=='R':
            x = copy.copy(R)
        elif param_to_plot=='neuron':
            x = copy.copy(n_neurons)
        elif param_to_plot=='layer':
            x = copy.copy(n_layer)
        
        ax.plot(x, score, '*', color='blue')
        if n_layer==2 and R==2:
            ax.plot(x, score, '*', color='red')
        if n_layer==2 and R==4:
            ax.plot(x, score, '*', color='green')
        if n_layer==2 and R==3:
            ax.plot(x, score, '*', color='orange')

        ax.set_xlabel(param_to_plot)
        ax.set_ylabel('accuracy')
        if score>score_previous:
            score_previous=score
            parameters_previous=parameters[ind]
    print(score_previous, parameters_previous)
    
def plot_accuracy_multi_param(scores, parameters):
    fig = plt.figure(figsize=(30,10))
    sub1 = fig.add_subplot(1,3,1, projection="3d")
    sub2 = fig.add_subplot(1,3,2, projection="3d")
    sub3 = fig.add_subplot(1,3,3, projection="3d")
    score_previous = 0
    colormap = plt.cm.get_cmap('summer_r', 10);
    for ind in range(len(scores)):
        score, lay, n_neuron, R = scores[ind], parameters[ind][0], parameters[ind][3], parameters[ind][1]
        sub1.scatter(n_neuron, lay, score, color=colormap(R/4), alpha=0.5)
        sub2.scatter(R, lay, score, color=colormap(n_neuron/16), alpha=0.5)
        sub3.scatter(n_neuron, R, score, color=colormap(lay/4), alpha=0.5)
        sub1.set_xlabel('neuron number')
        sub2.set_xlabel('R')
        sub3.set_xlabel('neuron number')
        sub1.set_ylabel('layer number')
        sub2.set_ylabel('layer number')
        sub3.set_ylabel('R')
        #sub1.view_init(10,-40)

In [ ]:
plot_accuracy(scores, parameters, 'tau')
plot_accuracy(scores, parameters, 'R')
plot_accuracy(scores, parameters, 'layer')
plot_accuracy(scores, parameters, 'neuron')